# <font color='blue'>Uni-Facef - DS - Regressão Linear com Pyspark </font>

Treinamento de um modelo com Regressão Linear usando PySpark

   O objetivo da Regressão Linear é prever os valores de uma variável dependente com base em resultados da variável independente. Então, da fórmula de regressão linear Y = ax + b, "x" é a variável independente (preditora) e y é a variável dependente (resposta), uma vez que "y" depende de "x". É um algoritmo de Aprendizado Supervisionado.

Baseado no artigo: https://www.cetax.com.br/blog/tutorial-pyspark-e-mllib/

In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .getOrCreate()
   
sc = spark.sparkContext

In [ ]:
df = spark.read \
    .option("delimiter", ",") \
    .csv('Salary_Data.csv', header=False)

df.show()

#### Renomeia os campos do dataframe

In [ ]:
df = df \
    .withColumnRenamed("_c0", "YearsExperience") \
    .withColumnRenamed("_c1", "Salary") \
    .select("Salary", "YearsExperience")

df.printSchema()

In [ ]:
from pyspark.sql.types import *


# Função para modifica o tipo de uma lista de colunas
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 


# Lista das colunas para coversão
columns = ['YearsExperience', 'Salary']


# Converte as colunas para "FloatType()"
df = convertColumn(df, columns, FloatType())

df.printSchema()

In [ ]:
df.show()

In [ ]:
df.groupBy("Salary").count().sort("Salary",ascending=False).show()

#### O método "describe()"

Faz a descrição do "df" baseado nas colunas, retornando uma contagem dos elementos, a média, desvio padrão, valores mínimo e máximo.

In [ ]:
df.describe().show()

#### Módulo DenseVector

O uso do DenseVector é uma maneira otimizada de lidar com valores numéricos, acelerando o processamento realizado pelo Spark.

In [ ]:
from pyspark.ml.linalg import DenseVector

# Define o RDD "input_data" para aplicar DenseVector 
input_data = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Replace `df` with the new DataFrame
df = spark.createDataFrame(input_data, ["label", "features"])

df.show()

####  Colocando os dados na mesma escala com método StantardScaler

O algoritmo de Regressão Linear trabalha com distâncias euclidianas, ele realiza operações de distância entre pontos no plano cartesiano. Isso exige que os dados estejam na mesma escala.

Com o StantardScaler, subtrai-se a média do valor "x" definido, e divide-se pela diferença (xmax – xmin). Desta forma, os dados estarão distribuidos ao longo de 0 na mesma escala.

In [ ]:
# Import `StandardScaler` 
from pyspark.ml.feature import StandardScaler


# Cria a instância do objeto
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit no DataFrame
scaler = standardScaler.fit(df)

# Transforma em dataframe com a escala
scaled_df = scaler.transform(df)

print(type(scaled_df))

# Inspect the result
scaled_df.take(2)

In [ ]:
help(scaled_df.take)

In [ ]:
scaled_df.show()

#### Slit em dados de treino e teste

In [ ]:
# Split the data into train and test sets
train_data, test_data = scaled_df.randomSplit([.75,.25],seed=1234)

train_data.show()

#### Instanciando o algoritmo e treinando o modelo (fit)

In [ ]:
# Import `LinearRegression`
from pyspark.ml.regression import LinearRegression

# Initialize `lr`
lr = LinearRegression(labelCol="label", maxIter=10)

# Fit the data to the model
linearModel = lr.fit(train_data)

type(linearModel)

#### Fazendo a predição com os dados de Treino

Criará a nova coluna "prediction" que dá para confrontar com a coluna "label" (variável resposta de treino)

In [ ]:
# Generate predictions
predicted = linearModel.transform(test_data)

predicted.show()

#### Coeficientes da equação

Visto que a equação é "Y = ax + b"

Onde "coefficients" é o valor de "a", e "intercept" é o valor de "b".

In [ ]:
# Coefficients para o modelo
linearModel.coefficients

In [ ]:
# Intercept para o modelo
linearModel.intercept

#### Erro Quadrático Médio, em inglês MSE, neste caso sendo representado pela raiz quadrada deste valor(RMSE)

In [ ]:
# Get the RMSE
linearModel.summary.rootMeanSquaredError

####  Métrica de avaliação do modelo

Extrai-se o R2, ou Coeficiente de Determinação, uma métrica estatística de proximidade de pontos e reta sobreposta

In [ ]:
linearModel.summary.r2

### Documentação

https://spark.apache.org/docs/latest/api/python/index.html

https://spark.apache.org/docs/latest/ml-classification-regression.html